## Install some dependencies

In [18]:
# install.packages('vegan')
# install.packages('ggrepel')
# install.packages('doParallel')
# install.packages('cowplot')

In [19]:
## These are the libraries FEAST needs
library("vegan")
library("dplyr")
library("doParallel")
library("foreach")
library("mgcv")
library("reshape2")
library("ggplot2")
library("cowplot")
library("Rcpp")
library("RcppArmadillo")

## `FEAST_main.R`

In [20]:
rm(list = ls())
gc()

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,1370222,73.2,2164898,115.7,2164898,115.7
Vcells,1524384,11.7,3902117,29.8,3902115,29.8


In [21]:
paste0("/home/jovyan/software/FEAST/FEAST_src/")

[1] "/home/jovyan/software/FEAST/FEAST_src/"

In [22]:
print("Change directory path")
dir_path = paste("/home/jovyan/software/FEAST/FEAST_src/")
setwd(dir_path)
source("src.R")

[1] "Change directory path"


In [23]:
#Set the arguments of your data
metadata_file = "your_metadata_file_name"
count_matrix = "your_taxa_count_matrix"
num_sources <- number_of_sources
EM_iterations = 1000 #default value

ERROR: Error in eval(expr, envir, enclos): object 'number_of_sources' not found


In [ ]:
setwd(paste0(dir_path, "Data_files"))

In [ ]:
# Load sample metadata
metadata <- read.csv(file = metadata_file ,header = T, sep = "\t", row.names = 1)

In [ ]:
# Load OTU table
otus <- read.table(file = count_matrix, header = T, comment = '', check = F, sep = '\t')
otus <- t(as.matrix(otus))

In [ ]:
# Extract only those samples in common between the two tables
common.sample.ids <- intersect(rownames(metadata), rownames(otus))
otus <- otus[common.sample.ids,]
metadata <- metadata[common.sample.ids,]
# Double-check that the mapping file and otu table
# had overlapping samples
if(length(common.sample.ids) <= 1) {
  message <- paste(sprintf('Error: there are %d sample ids in common '),
                   'between the metadata file and data table')
  stop(message)
}

envs <- metadata$Env

In [ ]:
# Extract the source environments and source/sink indices

train.ix <- which(metadata$SourceSink=='Source')
test.ix <- which(metadata$SourceSink=='Sink')
COVERAGE =  min(rowSums(otus[c(train.ix, test.ix),]))  #Can be adjusted by the user

In [ ]:
# Define sources and sinks

sources <- as.matrix(rarefy(otus[train.ix,], COVERAGE))
sinks <- as.matrix(rarefy(t(as.matrix(otus[test.ix,])), COVERAGE))

In [ ]:
print(paste("Number of OTUs in the sink sample = ",length(which(sinks > 0))))
print(paste("Seq depth in the sources and sink samples = ",COVERAGE))
print(paste("The sink is:", envs[test.ix]))

In [ ]:
# Estimate source proportions for each sink

FEAST_output<-FEAST(source=sources, sinks = t(sinks), env = envs[train.ix], em_itr = EM_iterations, COVERAGE = COVERAGE)
Proportions_est <- FEAST_output$data_prop[,1]
names(Proportions_est) <- c(as.character(envs[train.ix]), "unknown")

In [ ]:
print("Source mixing proportions")
Proportions_est